In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
import pickle

In [7]:
pip install --upgrade scikit-learn

In [9]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.5.0
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


In [17]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Fake news detection/train.csv'
df = pd.read_csv(path)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [18]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [20]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [21]:
df=df.fillna('')

In [22]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [23]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [ ]:
# Check the distribution of the target class
target_distribution = df['label'].value_counts()
print("Target class distribution:")
print(target_distribution)

target_distribution.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Distribution of Target Class')
plt.xticks(ticks=[0, 1], labels=['Real (0)', 'Fake (1)'], rotation=0)
plt.show()


Target class distribution:
label
1    10413
0    10387
Name: count, dtype: int64


In [ ]:
port_stem=PorterStemmer()
port_stem

In [ ]:
port_stem.stem("Hi thIs is nandana * % %@@@")

In [ ]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [ ]:
!pip install nltk
!python -m nltk.downloader stopwords

In [ ]:
stemming('Hi this is nandana')

In [ ]:
df['text']= df['text'].apply(stemming)

In [ ]:
x=df['text']

In [ ]:
y=df['label']

In [ ]:
y.shape

In [ ]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [ ]:
vect=TfidfVectorizer()

In [ ]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [ ]:
x_test.shape

In [ ]:
model=DecisionTreeClassifier()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
prediction=model.predict(x_test)

In [ ]:
prediction

In [ ]:
model.score(x_test, y_test)

In [ ]:
pickle.dump(vect, open('vector.pkl', 'wb'))
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [ ]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [ ]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [ ]:
val=fake_news("""In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  “If he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,” Jackie says. “Except the Hollywood left. They’ll get nauseous. ” “[Trump] could win the left over, they’ll fall in love with him in a minute. If he bombed them for a better reason,” Jackie explains. “Like if they have no transgender toilets. ” Jackie also says it’s no surprise that Hollywood celebrities didn’t support Trump’s strike on a Syrian airfield this month. “They were infuriated,” he says. “Because it might only save lives. That doesn’t mean anything to them. If it only saved the environment, or climate change! They’d be the happiest people in the world. ” Still, Jackie says he’s got nothing against Hollywood celebs. They’ve got a tough life in this country. Watch Jackie’s latest clip above.   Follow Daniel Nussbaum on Twitter: @dznussbaum """)

In [ ]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')


In [ ]:
from google.colab import files

files.download('model.pkl')

In [ ]:
from google.colab import files

files.download('vector.pkl')

In [ ]:
!pip install streamlit

In [ ]:

import streamlit as st

st.title('Fake News Classification app ')
st.subheader("Input the News content below")
sentence = st.text_area("Enter your news content here", "",height=200)
predict_btt = st.button("predict")
if predict_btt:
   prediction_class=fake_news(sentence)
   print(prediction_class)
   if prediction_class == [0]:
      st.success('Reliable')
    if prediction_class == [1]:
       st.warning('Unreliable')